This code is just a refernce for the modelling the train data. For more details check the the report, Readme and other directories 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

C:\Users\Dell\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\Dell\AppData\Local\Temp\ipykernel_7908\2011421623.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Reading the given train file 
file_path='D:/Competitions/SOI/AI-Clowns/training_datasets/train.csv'
df=pd.read_csv(file_path)
print(f"Shape:{df.shape}")
df.head()

Shape:(104409, 10)


,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1992,12,13,-4.99,156.0,4.8,-2.8,78.5,28.98,30.06
1,1989,5,14,0.00,-110.1,-2.5,2.4,NaN,24.95,25.23
2,1993,6,28,2.04,-110.2,-4.5,5.9,NaN,25.78,27.84
3,1995,1,8,1.98,-140.0,-7.7,1.2,82.6,26.47,26.86
4,1995,2,21,5.01,-170.0,-5.4,-6.6,75.2,28.83,28.90


In [3]:
# Sort the df in chronological order
df['date']=pd.to_datetime(df[['year','month','day']])
df=df.sort_values(by='date').reset_index(drop=True)
df.head()

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,date
0,1980,3,8,-0.02,-109.44,-4.9,1.1,NaN,25.66,25.97,1980-03-08
1,1980,3,9,-0.02,-109.44,-4.5,2.2,NaN,25.69,25.28,1980-03-09
2,1980,3,10,-0.02,-109.44,-3.8,1.9,NaN,25.56,24.31,1980-03-10
3,1980,3,12,-0.02,-109.44,-4.4,0.3,NaN,24.72,23.64,1980-03-12
4,1980,3,13,-0.02,-109.44,-3.2,0.1,NaN,24.66,24.34,1980-03-13


In [4]:
# Handling NULL values and saving it as another dataset called updated.csv (done in other files)
# Handling NULL values

print(f"NULL values before interpolation\n {df.isnull().sum()}")

# List of features to interpolate
features_to_interpolate = ['air temp.', 'zon.winds', 'mer.winds','humidity']

# Apply spline interpolation to each feature
for feature in features_to_interpolate:
    df[feature] = df[feature].interpolate(method='spline', order=3)

print(f"Null values after interpolation\n {df.isnull().sum()}")

# To handle edge values of humidty use XGboost
from sklearn.model_selection import train_test_split,GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split data into two sets: one with NaN values and one without
df_known = df.dropna(subset=['humidity'])  # DataFrame without NaN values
df_nan = df[df['humidity'].isna()]         # DataFrame with NaN values

# Features and target variable
X = df_known[['month', 'year','s.s.temp.','latitude','longitude','air temp.']]
y = df_known['humidity']

param_grid = {
    'n_estimators': [400, 500, 600],  # Number of trees
    'max_depth': [7, 9, 11],           # Maximum depth of each tree
    'learning_rate': [0.5, 0.1, 0.5], # Learning rate
    'subsample': [0.7, 0.9, 1.1],      # Subsample ratio of the training instances
    'colsample_bytree': [0.9, 1.0, 2.0],# Subsample ratio of columns when constructing each tree
}

# Create the XGBoost regressor
model = XGBRegressor()

# Instantiate the grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Perform grid search
grid_search.fit(X, y)

# Best parameters found during grid search
best_params = grid_search.best_params_

# Predict on the test set with the best model
best_model = grid_search.best_estimator_
imputed_humidity = best_model.predict(df_nan[['month', 'year','time_elapsed','s.s.temp.','latitude','longitude','air temp.']])
df.loc[df['humidity'].isna(), 'humidity'] = imputed_humidity

print(f"Null values after XGB\n {df.isnull().sum()}")

'# Handling NULL values\n\nprint(f"NULL values before interpolation\n {df.isnull().sum()}")\n\n# List of features to interpolate\nfeatures_to_interpolate = [\'air temp.\', \'zon.winds\', \'mer.winds\',\'humidity\']\n\n# Apply spline interpolation to each feature\nfor feature in features_to_interpolate:\n    df[feature] = df[feature].interpolate(method=\'spline\', order=3)\n\nprint(f"Null values after interpolation\n {df.isnull().sum()}")\n\n# To handle edge values of humidty use XGboost\nfrom sklearn.model_selection import train_test_split,GridSearchCV\nfrom xgboost import XGBRegressor\nfrom sklearn.metrics import mean_squared_error, r2_score\n\n# Split data into two sets: one with NaN values and one without\ndf_known = df.dropna(subset=[\'humidity\'])  # DataFrame without NaN values\ndf_nan = df[df[\'humidity\'].isna()]         # DataFrame with NaN values\n\n# Features and target variable\nX = df_known[[\'month\', \'year\',\'s.s.temp.\',\'latitude\',\'longitude\',\'air temp.\']]\ny = 

In [5]:
df=pd.read_csv('D:/Competitions/SOI/AI-Clowns/training_datasets/updated.csv')
df.drop(columns=['time_elapsed'],inplace=True) # This column was required for splitting data during model selection phase 
df.drop(columns=['day'],inplace=True) # feature selection 
print(f"NULL values of the updated csv :\n{df.isnull().sum()}")

NULL values of the updated csv :
year         0
month        0
latitude     0
longitude    0
zon.winds    0
mer.winds    0
humidity     0
air temp.    0
s.s.temp.    0
dtype: int64


In [6]:
#Apply certain trasformations

from sklearn.preprocessing import StandardScaler,FunctionTransformer,PowerTransformer

X=df.drop(columns=['s.s.temp.'])
y=df['s.s.temp.']

yeojohnson_transformer = PowerTransformer(method='yeo-johnson')
X['zon.winds'] = yeojohnson_transformer.fit_transform(X[['zon.winds']])
X['air temp.']=yeojohnson_transformer.fit_transform(X[['air temp.']])

scaler=StandardScaler()
X=scaler.fit_transform(X)

In [7]:
# Training the model and saving the parameters

import xgboost as xgb
import pickle

# Best parameters from grid search of model selection file
best_params = {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 400}

# Initialize the XGBoost classifier with the best parameters
model = xgb.XGBClassifier(**best_params)

# Train the model on the entire dataset
model.fit(X, y)

# Save the trained model using pickle
with open('xgb_final_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model trained and saved successfully using pickle.")


'import xgboost as xgb\nimport pickle\n\n\n# Best parameters from grid search\nbest_params = {\'learning_rate\': 0.05, \'max_depth\': 5, \'n_estimators\': 400}\n\n# Initialize the XGBoost classifier with the best parameters\nmodel = xgb.XGBClassifier(**best_params)\n\n# Train the model on the entire dataset\nmodel.fit(X, y)\n\n# Save the trained model using pickle\nwith open(\'xgb_final_model.pkl\', \'wb\') as f:\n    pickle.dump(model, f)\n\nprint("Model trained and saved successfully using pickle.")'